
# Opinion‑BERT Microservice (Prototype) — FastAPI + OpenAPI on Colab
This notebook spins up a **FastAPI** microservice (with OpenAPI docs) that mimics the API surface for an Opinion‑BERT style model (multi‑task: sentiment + mental‑health status).  
It uses **DistilBERT** for sentiment + **TextBlob/VADER** for opinion features as a **baseline**. You can later swap in your Hybrid BERT + CNN + BiGRU model.

**Endpoints:**
- `GET /health` — liveness check  
- `GET /ready` — readiness + model load time  
- `POST /sentiment/analyze` — main inference (sentiment + status + opinion features)

> ⚠️ This is **not medical advice** and **not a diagnostic tool**. It’s a research prototype for software integration only.


In [ ]:

# ✅ Step 1: Install dependencies
# (FastAPI, Uvicorn, Transformers, PyTorch, TextBlob, VADER, Cloudflared, etc.)
!pip -q install fastapi "uvicorn[standard]" transformers torch textblob vaderSentiment nest-asyncio starlette >/dev/null

# Cloudflared (for public URL without auth)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!sudo dpkg -i cloudflared-linux-amd64.deb >/dev/null 2>&1 || true

# Optional (sometimes helpful): download TextBlob corpora
# !python -m textblob.download_corpora
print("✅ Dependencies installed.")


In [ ]:

# ✅ Step 2: Create the FastAPI app (app.py)
%%writefile app.py
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
import time

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline

APP_TITLE = "Opinion‑BERT Microservice (Prototype)"
APP_VERSION = "0.1.0"
APP_DESC = (
    "Baseline service exposing an Opinion‑BERT‑style API surface with OpenAPI. "
    "Implements sentiment + status (heuristic) + opinion features. Replace the model later."
)

app = FastAPI(title=APP_TITLE, version=APP_VERSION, description=APP_DESC)

# CORS (open for demo; restrict for production)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

class AnalyzeRequest(BaseModel):
    text: str = Field(..., min_length=1, max_length=4000)
    context: Optional[List[str]] = Field(default=None, description="Optional previous messages for context.")
    return_explanations: bool = Field(default=False, description="Reserved for saliency/attention outputs later.")

class TaskScore(BaseModel):
    label: str
    score: float

class OpinionFeatures(BaseModel):
    polarity: float
    subjectivity: float
    vader: Dict[str, float]

class AnalyzeResponse(BaseModel):
    sentiment: TaskScore
    status: TaskScore
    opinion: OpinionFeatures
    timings_ms: Dict[str, float]
    model: Dict[str, str]

@app.on_event("startup")
def load_model():
    # Load baseline components
    start = time.time()
    app.state.sentiment = pipeline(
        "sentiment-analysis",
        model="distilbert-base-uncased-finetuned-sst-2-english"
    )
    app.state.vader = SentimentIntensityAnalyzer()
    app.state.load_time_ms = int((time.time() - start) * 1000)

@app.get("/health", tags=["ops"])
def health():
    return {"status": "ok"}

@app.get("/ready", tags=["ops"])
def ready():
    return {"ready": True, "load_time_ms": getattr(app.state, "load_time_ms", None)}

@app.post("/sentiment/analyze", response_model=AnalyzeResponse, tags=["inference"])
def analyze(req: AnalyzeRequest):
    t0 = time.time()
    tb = TextBlob(req.text)
    pol = float(tb.sentiment.polarity)
    sub = float(tb.sentiment.subjectivity)
    t_tb = (time.time() - t0) * 1000

    t1 = time.time()
    vs = app.state.vader.polarity_scores(req.text)
    t_vader = (time.time() - t1) * 1000

    t2 = time.time()
    out = app.state.sentiment(req.text)[0]
    # Normalize labels to lowercase "positive"/"negative"
    label = "positive" if "POS" in out["label"].upper() else "negative"
    conf = float(out["score"])
    t_hf = (time.time() - t2) * 1000

    # Heuristic "status" head to mimic multi-task output (replace with your trained head later)
    if label == "negative" or pol < -0.2 or vs["compound"] < -0.35:
        status_label = "at_risk"
        # naive score: higher when negativity cues increase
        status_score = min(1.0, max(0.0, 0.5 + (abs(min(0, pol)) + abs(min(0, vs['compound']))) / 1.5))
    else:
        status_label = "well"
        status_score = conf

    return AnalyzeResponse(
        sentiment=TaskScore(label=label, score=conf),
        status=TaskScore(label=status_label, score=float(status_score)),
        opinion=OpinionFeatures(polarity=pol, subjectivity=sub, vader=vs),
        timings_ms={"textblob": t_tb, "vader": t_vader, "hf": t_hf},
        model={
            "encoder": "distilbert-base-uncased-finetuned-sst-2-english",
            "service_version": APP_VERSION,
            "note": "Heuristic multi-task head. Swap with Hybrid BERT+CNN+BiGRU later."
        },
    )
print("✅ Wrote app.py")


In [ ]:

# ✅ Step 3: Launch the FastAPI app (Uvicorn) in the background
import nest_asyncio, uvicorn, threading, time
nest_asyncio.apply()

def _run():
    uvicorn.run("app:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=_run, daemon=True)
thread.start()
time.sleep(2)
print("✅ Uvicorn started on http://127.0.0.1:8000  (next cell will create a public URL)")


In [ ]:

# ✅ Step 4: Expose the API publicly (no account needed)
import re, time, os, subprocess, pathlib

# Start Cloudflared tunnel in background and log output to file
log_path = "cf.log"
if os.path.exists(log_path):
    os.remove(log_path)

proc = subprocess.Popen(
    ["cloudflared", "tunnel", "--url", "http://localhost:8000", "--no-autoupdate"],
    stdout=open(log_path, "w"),
    stderr=subprocess.STDOUT,
    text=True
)

# Wait for the public URL to appear
public_url = None
for _ in range(30):  # ~30 * 0.5s = 15s max wait
    time.sleep(0.5)
    if os.path.exists(log_path):
        txt = open(log_path).read()
        m = re.search(r"https://[a-zA-Z0-9-]+\.trycloudflare\.com", txt)
        if m:
            public_url = m.group(0)
            break

if public_url:
    print("🌍 Public URL:", public_url)
    print("📚 OpenAPI docs:", public_url + "/docs")
else:
    print("⚠️ Could not detect public URL. Scroll up to the cell output; Cloudflared prints it early.")


In [ ]:

# ✅ Step 5: Smoke test (change `PUBLIC` if needed)
import json, requests

try:
    PUBLIC = public_url  # from previous cell
except NameError:
    PUBLIC = "http://127.0.0.1:8000"

print("Testing against:", PUBLIC)

health = requests.get(PUBLIC + "/health", timeout=10).json()
ready = requests.get(PUBLIC + "/ready", timeout=10).json()
demo = requests.post(PUBLIC + "/sentiment/analyze",
                     json={"text": "I feel hopeless and exhausted. Nothing seems to help."},
                     timeout=15).json()

print("GET /health ->", health)
print("GET /ready  ->", ready)
print("POST /sentiment/analyze ->")
print(json.dumps(demo, indent=2))



## 🔄 Swap in your Hybrid BERT + CNN + BiGRU (later)
- Replace the `pipeline("sentiment-analysis", ...)` with your **BERT encoder**.
- Concatenate **opinion embeddings** computed from the text (or integrate as cross-features).
- Add **CNN + BiGRU** layers and **two heads**: `sentiment` and `status`.
- Keep the same **Pydantic request/response** models so the API remains stable.
- Use **MLflow** to version models and **Redis** to cache hot predictions.
- Expose model metadata under `model` in the response.
